# Script Demo
Show how to use the provided scripts that help with creating expertiment designs and processing the results

## Creating Experiment Design

### Cross Product Experiment Design (from table format)
Converts an experiment config in `table` form (concise) into the experiment design form required for the experiment suite.     
The function builds a cartesian product of all configuration options marked as `$FACTOR$` with a list of levels.    
The experiment in table form is a yaml file (with name `<exp_name>.yml` in `experiments/table`).
Each factor of the experiment is a yaml object with a single entry with key `$FACTOR$` and as a value the list of levels.

See example `experiments/table/demo.yml`and the resulting `experiments/designs/demo.yml`.

In [26]:
#%pycat scripts/expdesign.py # uncomment to see the code
%run scripts/expdesign.py --exp_suite demo --exps demo_exp1 demo_exp2

with open("experiments/designs/demo.yml", "r") as f:
    experiments_config = yaml.load(f, Loader=yaml.SafeLoader)
display(experiments_config)

Writing experiment design to: experiments/designs/demo.yml



{'experiments': [{'demo_exp1': {'n_repetitions': 2,
    'common_roles': ['setup-common'],
    'host_types': {'client': {'n': 1, 'init_role': 'setup-client'},
     'server': {'n': 1, 'check_status': False, 'init_role': 'setup-server'}},
    'base_experiment': {'w': 13,
     'x': '$FACTOR$',
     'host_vars': {'client': {'y': 'abc'}, 'server': {'z': '$FACTOR$'}}},
    'factor_levels': [{'x': 4, 'host_vars': {'server': {'z': 1}}},
     {'x': 4, 'host_vars': {'server': {'z': 2}}},
     {'x': 5, 'host_vars': {'server': {'z': 1}}},
     {'x': 5, 'host_vars': {'server': {'z': 2}}},
     {'x': 6, 'host_vars': {'server': {'z': 1}}},
     {'x': 6, 'host_vars': {'server': {'z': 2}}},
     {'x': 7, 'host_vars': {'server': {'z': 1}}},
     {'x': 7, 'host_vars': {'server': {'z': 2}}}]}},
  {'demo_exp2': {'n_repetitions': 2,
    'common_roles': ['setup-common'],
    'host_types': {'client': {'n': 1, 'init_role': 'setup-client'},
     'server': {'n': 1, 'init_role': 'setup-server'}},
    'base_experim

## Reading and Processing Results

In [21]:
import pandas as pd

results_dir = "results"

In [23]:
# A dict with all experiment names and for each a list of experiment ids that should be included in the dataframe.
# See `scripts/results.py` to adjust how different output files are used.  
#%pycat scripts/results.py # uncomment to see the code
%run scripts/results.py

exp = {
    "example" : ["1628751545"]
}

df = read_df(results_dir, exp)
display(df)

,exp_suite_id,exp_name,run,rep,host_type,host_idx,info,host_vars.client.arg1,host_vars.client.arg2,host_vars.server.arg3,host_vars.server.arg4,a0,a1,a2,a3,a4,a5,tp
0,1628751545,experiment_1,run_2,rep_0,server,0,c3,10,3,3,test1,-config,config.json,-a3,3,-a4,test1,7
1,1628751545,experiment_1,run_2,rep_0,client,0,c3,10,3,3,test1,-config,config.json,-a1,10,-a2,3,17
2,1628751545,experiment_1,run_1,rep_0,server,0,c2,10,2,2,test1,-config,config.json,-a3,2,-a4,test1,1
3,1628751545,experiment_1,run_1,rep_0,client,0,c2,10,2,2,test1,-config,config.json,-a1,10,-a2,2,10
4,1628751545,experiment_1,run_0,rep_0,server,0,c1,10,1,1,test1,-config,config.json,-a3,1,-a4,test1,5
5,1628751545,experiment_1,run_0,rep_0,client,0,c1,10,1,1,test1,-config,config.json,-a1,10,-a2,1,7
6,1628751545,experiment_2,run_0,rep_0,server,0,c2,10,2,2,test1,-config,config.json,-a3,2,-a4,test1,8
7,1628751545,experiment_2,run_0,rep_0,client,0,c2,10,2,2,test1,-config,config.json,-a1,10,-a2,2,9


### Group By (Aggregate Repetitions)
The experiment repeats each run `r` times.
Here we show how to apply this group by aggregate transformation on the dataframe to calculate mean, min, max, and std dev across repetitions of the same run.
(for clarity, we exclude the other config options but they could be used as further group by columns) 

In [24]:
# Example 1: Only keep the main info

df1 = df

# sometimes (e.g., for csv result files) it's necessary to explicitly specify which columns are numbers
df1[["host_vars.client.arg1", "tp"]] = df1[["host_vars.client.arg1", "tp"]].apply(pd.to_numeric)


df1 = df1.groupby(['exp_name', 'exp_suite_id', 'run', 'host_type']).agg({'tp': ['mean', 'min', 'max', 'std']}).reset_index()

df1.columns = ["_".join(v) if v[1] else v[0] for v in df1.columns.values]

display(df1)

,exp_name,exp_suite_id,run,host_type,tp_mean,tp_min,tp_max,tp_std
0,experiment_1,1628751545,run_0,client,7.0,7,7,NaN
1,experiment_1,1628751545,run_0,server,5.0,5,5,NaN
2,experiment_1,1628751545,run_1,client,10.0,10,10,NaN
3,experiment_1,1628751545,run_1,server,1.0,1,1,NaN
4,experiment_1,1628751545,run_2,client,17.0,17,17,NaN
5,experiment_1,1628751545,run_2,server,7.0,7,7,NaN
6,experiment_2,1628751545,run_0,client,9.0,9,9,NaN
7,experiment_2,1628751545,run_0,server,8.0,8,8,NaN


In [25]:
# Example 2: Keep all config info
cols = df.columns.tolist()
cols.remove('rep') # remove rep because we want to aggregate over the reps
cols.remove('tp') # remove the `value` column `tp` (throughput)

df1 = df.groupby(cols).agg({'tp': ['mean', 'min', 'max', 'std']}).reset_index()
df1.columns = ["_".join(v) if v[1] else v[0] for v in df1.columns.values]
display(df1)

,exp_suite_id,exp_name,run,host_type,host_idx,info,host_vars.client.arg1,host_vars.client.arg2,host_vars.server.arg3,host_vars.server.arg4,a0,a1,a2,a3,a4,a5,tp_mean,tp_min,tp_max,tp_std
0,1628751545,experiment_1,run_0,client,0,c1,10,1,1,test1,-config,config.json,-a1,10,-a2,1,7.0,7,7,NaN
1,1628751545,experiment_1,run_0,server,0,c1,10,1,1,test1,-config,config.json,-a3,1,-a4,test1,5.0,5,5,NaN
2,1628751545,experiment_1,run_1,client,0,c2,10,2,2,test1,-config,config.json,-a1,10,-a2,2,10.0,10,10,NaN
3,1628751545,experiment_1,run_1,server,0,c2,10,2,2,test1,-config,config.json,-a3,2,-a4,test1,1.0,1,1,NaN
4,1628751545,experiment_1,run_2,client,0,c3,10,3,3,test1,-config,config.json,-a1,10,-a2,3,17.0,17,17,NaN
5,1628751545,experiment_1,run_2,server,0,c3,10,3,3,test1,-config,config.json,-a3,3,-a4,test1,7.0,7,7,NaN
6,1628751545,experiment_2,run_0,client,0,c2,10,2,2,test1,-config,config.json,-a1,10,-a2,2,9.0,9,9,NaN
7,1628751545,experiment_2,run_0,server,0,c2,10,2,2,test1,-config,config.json,-a3,2,-a4,test1,8.0,8,8,NaN
